In [1]:
import pandas as pd

In [2]:
df = pd.read_table('./data/ai.ja.txt.parsed', sep='\t|,', header=None, usecols=[0, 1, 2, 7], names=['surface', 'pos', 'pos1', 'base'], skiprows=2)
df_with_eos = df[(df['pos'].isna() == False) | (df['surface'] == 'EOS')] # cabochaによる形態素解析は、若干EOSの入れ方が誤っている気がするが、気にしないことにする。

/Users/hiroga/.ghq/github.com/xhiroga/til/computer-science/nlp/nlp100/.venv/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
# 40. 係り受け解析結果の読み込み（形態素）
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def __str__(self):
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'.format(self.surface, self.base, self.pos, self.pos1)

In [4]:
from pandas import Series
def getSentenseId(s: Series) -> Series:
    return (s.shift() == 'EOS').cumsum()

df_with_eos['sentense_id'] = getSentenseId(df_with_eos['surface'])

/var/folders/vt/s15n0wxx10zgxq87t9mrzgqh0000gp/T/ipykernel_59000/2924014788.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_eos['sentense_id'] = getSentenseId(df_with_eos['surface'])


In [5]:
morphs = df_with_eos.groupby('sentense_id').apply(lambda x: [Morph(row['surface'], row['base'], row['pos'], row['pos1']) for index, row in x.iterrows()])

In [6]:
# 41. 係り受け解析結果の読み込み（文節・係り受け）
class Chunk():
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.dst = dst
        self.srcs = []

    def __str__(self):
        return 'morphs[{}]\tdst[{}]\tsrcs[{}]'.format(self.morphs, self.dst, self.srcs)

In [13]:
def getChunkId(s: Series) -> Series:
    return (s.isna()).cumsum()

df['sentense_id'] = getSentenseId(df['surface'])
df['chunk_id'] = getChunkId(df['pos'])


In [ ]:
from pandas import DataFrame

def sentenseToArrayOfChunk(sentense_df: DataFrame):
    sentense_df.groupby('chunk_id').apply(lambda chunk: Chunk(chunkToArrayOfMorph(chunk), chunk['dst']))

def chunkToArrayOfMorph(chunk_df: DataFrame):
    chunk_df.apply(lambda morph: Morph(morph['surface'], morph['base'], morph['pos'], morph['pos1']))